In [1]:
import seldon.pipeline.auto_transforms as auto
import seldon.pipeline.util as sutl
import seldon.pipeline.cross_validation as cf
from sklearn.pipeline import Pipeline
import pandas as pd
from seldon import xgb
import seldon.pipeline.basic_transforms as bt

df = pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data",
                 header=None,names=["A1","A2","A3","A4","A5","A6","A7","A8","A9","A10","A11","A12","A13","A14","A15","target"])

In [2]:
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,target
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [19]:
tTargetId = bt.Feature_id_transform(zero_based=True,input_feature="target",output_feature="targetId")
t_auto = auto.Auto_transform(exclude=["target","targetId"])
xg = xgb.XGBoostClassifier(target="targetId",excluded=["target"])
cv = cf.Seldon_KFold(xg,10)
transformers = [("tId",tTargetId),("tAuto",t_auto),("xgb",cv)]
p = Pipeline(transformers)
p.fit_transform(df)
print "avg cross validation accuracy ",cv.get_score()

Estimator init called
fitting scaler for col  A3
fitting scaler for col  A8
fitting scaler for col  A11
fitting scaler for col  A15
num columns to drop  0
num scalers 4
num categorical  11
num dates 0
num date diffs 0
num bool 0
convert categorical  A1 1 / 11
convert categorical  A2 2 / 11
convert categorical  A4 3 / 11
convert categorical  A5 4 / 11
convert categorical  A6 5 / 11
convert categorical  A7 6 / 11
convert categorical  A9 7 / 11
convert categorical  A10 8 / 11
convert categorical  A12 9 / 11
convert categorical  A13 10 / 11
convert categorical  A14 11 / 11
scaling col  A15 1 / 4
scaling col  A8 2 / 4
scaling col  A3 3 / 4
scaling col  A11 4 / 4
excluding features ['target']
{'reg_alpha': 0, 'colsample_bytree': 1, 'silent': True, 'colsample_bylevel': 1, 'scale_pos_weight': 1, 'learning_rate': 0.1, 'missing': None, 'max_delta_step': 0, 'nthread': -1, 'base_score': 0.5, 'n_estimators': 100, 'subsample': 1, 'reg_lambda': 1, 'seed': 0, 'min_child_weight': 1, 'objective': 'reg:l

In [22]:
from seldon.pipeline import bayes_optimize as bopt
e_opt = bopt.BayesOptimizer(xg,{'learning_rate': (0.0001, 0.1),'n_estimators': (10,1000),'max_depth':(3,20)},
                            param_int=['n_estimators','max_depth'],cv_folds=5)
transformers = [("tId",tTargetId),("tAuto",t_auto),("bopt",e_opt)]
p = Pipeline(transformers)
p.fit_transform(df)
print "Best accuracy is ",e_opt.get_best_score()

fitting scaler for col  A3
fitting scaler for col  A8
fitting scaler for col  A11
fitting scaler for col  A15
num columns to drop  0
num scalers 4
num categorical  44
num dates 0
num date diffs 0
num bool 0
convert categorical  A1 1 / 44
convert categorical  A2 2 / 44
convert categorical  A4 3 / 44
convert categorical  A5 4 / 44
convert categorical  A6 5 / 44
convert categorical  A7 6 / 44
convert categorical  A9 7 / 44
convert categorical  A10 8 / 44
convert categorical  A12 9 / 44
convert categorical  A13 10 / 44
convert categorical  A14 11 / 44
convert categorical  A1 12 / 44
convert categorical  A2 13 / 44
convert categorical  A4 14 / 44
convert categorical  A5 15 / 44
convert categorical  A6 16 / 44
convert categorical  A7 17 / 44
convert categorical  A9 18 / 44
convert categorical  A10 19 / 44
convert categorical  A12 20 / 44
convert categorical  A13 21 / 44
convert categorical  A14 22 / 44
convert categorical  A1 23 / 44
convert categorical  A2 24 / 44
convert categorical  A4 25